In [1]:
import logging
logging.captureWarnings(False)
logging.getLogger('py.warnings').setLevel(logging.ERROR)
from dask.distributed import Client, progress
client = Client(n_workers=5, threads_per_worker=4, memory_limit='100GB')
client

/Users/bkirtman/miniconda3/envs/myenv/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63378 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:63378/status,
Dashboard: http://127.0.0.1:63378/status,Workers: 5
Total threads: 20,Total memory: 465.66 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63379,Workers: 5
Dashboard: http://127.0.0.1:63378/status,Total threads: 20
Started: Just now,Total memory: 465.66 GiB
Comm: tcp://127.0.0.1:63397,Total threads: 4
Dashboard: http://127.0.0.1:63398/status,Memory: 93.13 GiB
Nanny: tcp://127.0.0.1:63382,


In [2]:
import numpy as np
import torch
import subprocess
import sys
import torch_harmonics as th
import torch_harmonics.distributed as dist
import pandas as pd
import matplotlib.pyplot as plt
from subs1_utils import *

In [3]:
# For MacOS is higher than 12.3+
if torch.backends.mps.is_available():
    print("Running on GPU")
    device = torch.device("mps")
    print("MPS is activated:",torch.backends.mps.is_built()) # Was the current version of PyTorch built with MPS activated?
else:
    print("Running  on CPU")
    device = torch.device("cpu")

device

Running on GPU
MPS is activated: True


device(type='mps')

In [4]:
### This cell initializes the model
###
###     First Define all spectral grids
###
zw = 63 # zonal wave number
mw = 63 # meridional wave number
kmax = 26
imax = 192
jmax = 96
steps_per_day = 216*1.5 ### Changing this number impliws time step changes and should
#                       be implemented carefully
###
spec = (mw,zw,kmax)
grid = (imax,jmax,kmax)
#
#
# provide experiment name and data path for writing out data
# datapath may need to be edited for your system
#
expname = 'TestT63L26_Dist'
foo = str(subprocess.check_output(['whoami']))
end = len(foo) - 3
uname = foo[2:end]
datapath = '/Users/'+uname+'/Work/AGCM/AGCM/tmp4/'+expname+'/'
toffset = 0
#
# If restart see below settings
#
#toffset = 600*1 # toffset is the number of days that have already run
#datapath_init = '/Users/'+uname+'/Work/AGCM/AGCM/tmp4/Restarts/Total/' # Location in restarts
datapath_init = datapath #set equal to datapath if restarting in same directory
#
if ( toffset == 0): # Cold Start
    subprocess.call(['rm','-r', datapath])
    subprocess.check_output(['mkdir', datapath])

In [5]:
# Get the Gaussian latitudes and equally spaced longitudes
#
cost_lg, wlg, lats = precompute_latitudes(jmax)
lats = 90-180*lats/(np.pi)
lons = np.linspace(0.0,360.0-360.0/imax,imax)
#
# Instantiate grid to spectral (dsht) and spectral to grid (disht) distibuted transforms
#
vsht = th.RealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
dsht = dist.DistributedRealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
disht = dist.DistributedInverseRealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
dvsht = dist.DistributedRealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
divsht = dist.DistributedInverseRealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)

In [6]:
#
#
#
# Initialize spectral fields (at rest or to be read in)
def initialize(temp_newton,lnpsclim,kmax,mw,zw):
    for k in range (kmax):
        tmn1[k] = tmn1[k] + temp_newton[k]
        tmn2[k] = tmn2[k] + temp_newton[k]
        tmn3[k] = tmn3[k] + temp_newton[k]
    qmn1 = lnpsclim
    qmn2 = lnpsclim
    qmn3 = lnpsclim
    return tmn1,tmn2,tmn3,qmn1,qmn2,qmn3

In [7]:
#
# Initialization: Could read spectral restarts, or could
#              start at rest. If wanting to use grid point see
#              jupyter notebook preprocess
#              
#
# Implement at rest initial condition, but need coriolis since
# model predicts total vorticity
#
coriolis = np.broadcast_to([(4.0*np.pi/86400)*np.sin(lats[j,np.newaxis]*np.pi/180.0) for j in range(jmax)], (jmax, imax))
#
#
# Initialize spectral fields (at rest or to be read in)
# Need to read in background temperature data for Newtonian
# Relaxation and possible initialization, see preprocess
# for how to change source data or formulation.
#
temp_newton = torch.load('temp.spectral.pt') # newtonian 
#                                    relaxation temperature, spectral
lnpsclim = torch.load('lnps.spectral.pt') # lnps climatology for damping
#
# Read Climatology on Gausian Grid
#
tclim_gg = torch.load('tsig.ggrid.pt')
vortclim_gg = torch.load('vortsig.ggrid.pt')
divclim_gg = torch.load('divsig.ggrid.pt')
#
divclim = dsht(divclim_gg)
vortclim = dsht(vortclim_gg)
tclim = dsht(tclim_gg)
#
zmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
zmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    zmn1 = torch.load(datapath_init+'zmn1.spectral.pt')
    zmn2 = torch.load(datapath_init+'zmn2.spectral.pt')
zmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
dmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128) 
dmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    dmn1 = torch.load(datapath_init+'dmn1.spectral.pt')
    dmn2 = torch.load(datapath_init+'dmn2.spectral.pt')
dmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
tmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
tmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    tmn1 = torch.load(datapath_init+'tmn1.spectral.pt') 
    tmn2 = torch.load(datapath_init+'tmn2.spectral.pt')
tmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
wmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
qmn1 = torch.zeros((mw,zw),dtype=torch.complex128)
qmn2 = torch.zeros((mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    qmn1 = torch.load(datapath_init+'qmn1.spectral.pt')
    qmn2 = torch.load(datapath_init+'qmn2.spectral.pt')
qmn3 = torch.zeros((mw,zw),dtype=torch.complex128)
#
#
if ( toffset == 0 ): # Only do this if cold start
    tmn1,tmn2,tmn3,qmn1,qmn2,qmn3 =\
    initialize(tclim,lnpsclim,kmax,mw,zw)
#
# Topography data - this should be spectral data or can be
#                        initialized to zero. If grid point data
#                        is desired see preprocess for how to
#                        convert to spectral. 
#
# Setting topography to zero here
#
###phismn = torch.zeros((mw,zw),dtype=torch.complex128)
#
# If non-zero topog read here
#
phismn = torch.load('topog.spectral.pt')
#
#
# Adding heating here see preprocess.ipynb
#
####heat = torch.load('heat.ggrid.pt')
#
# or set to zero
#
heat = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
#

In [8]:
###
### Constants, parameters, vertical differencing parameters,
### matricies for geopotential height, etc ...
###
delsig, si, sl, sikap, slkap, cth1, cth2, r1b, r2b = bscst(kmax)
### The above code is in subs1_utils.py - vertical structure related
### This code would need to be changed if the vertical resolution
### is changed - could be done by simply specifying delsig in bscst
###
amtrx, cmtrx, dmtrx = mcoeff(kmax,si,sl,slkap,r1b,r2b,delsig)
### The above code is for geopotential height and implicit scheme
### in subs1_utils.py but unlikely any changes would be needed
emtrx = inv_em(dmtrx,steps_per_day,kmax,mw,zw)
### The above code
### emtrix is used in the implicit time scheme, computed once here to save cpu time
### changes unlikely

In [9]:
#### Preprocessing is complete - now time to run model
#
#
# The Model Runs in 30-day chuncks - need to specify how many 30-day chunks to run
tl = 30 ##### tl is the chunk size - typically 30 days, but for testing 3 is reasonable
#
# Suggested ichunk for time dependent models: 120
#
ae = 6.371E+06 # Earth radius
tmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
zmnt = tmnt.detach().clone()
dmnt = tmnt.detach().clone()
qmnt = torch.zeros((tl,mw,zw),dtype=torch.complex128)
wmnt = tmnt.detach().clone()
#
ddtdiv = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
ddtvort = ddtdiv.detach().clone()
ttend = ddtdiv.detach().clone()
#
vort = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
div = ddtdiv .detach().clone()
temp = ddtdiv.detach().clone()
qdot = ddtdiv.detach().clone()
#
times = pd.date_range(start = '1950-01-01', end='2100-01-01', freq='D')
# 
ichunk = 20
#
idays = tl * ichunk
#
#
#
#
# Begin Time Loop
#
ii = 0
savedat = 0
daycount = 0
total_days = 0
nstep = idays*steps_per_day
while ii < nstep:
    ii = ii + 1
    savedat = savedat + 1
    zmnt[daycount] = zmnt[daycount] + zmn1/steps_per_day
    dmnt[daycount] = dmnt[daycount] + dmn1/steps_per_day
    tmnt[daycount] = tmnt[daycount] + tmn1/steps_per_day
    qmnt[daycount] = qmnt[daycount] + qmn1/steps_per_day
    wmnt[daycount] = wmnt[daycount] + wmn1/steps_per_day
    if (savedat == steps_per_day): # post processing
        #
        # Call Postprocessing Routine as needed
        #
        print((dmn1[10,2,2],dmn1[9,2,1]))
        daycount = daycount + 1
        total_days = total_days + 1
        print(['Day = ',total_days])
        if (daycount == tl):
            times_30day = times[total_days-tl+toffset:total_days+toffset]
            postprocessing(disht,divsht,zmnt,dmnt,tmnt,qmnt,wmnt,\
                           phismn,amtrx,times_30day,mw,zw,\
                           kmax,imax,jmax,sl,lats,lons,tl,datapath)
            tmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            zmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            dmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            qmnt = torch.zeros((tl,mw,zw),dtype=torch.complex128)
            wmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            daycount = 0
        savedat = 0
    #
    # Run model for one time step
    #
    # Spectral to grid transformation of needed fields:
    #   Vorticity, divergence, temperature, U, V, 
    #   grad(ln(Ps)), Q prescibed heating
    #
    #
    vort = disht(zmn2) ### This is the relative vorticity
    div = disht(dmn2)
    temp = disht(tmn2)
    qdot = disht(wmn2)
    u,v = uv(divsht,zmn2,dmn2,mw,zw,kmax,imax,jmax)
    dxq,dyq = gradq(divsht,qmn2,mw,zw,imax,jmax)
    #
    # Non-Linear products
    #
    a,b,e,ut,vt,ri,wj,cbar,dbar = nlprod(u,v,vort,div,temp,dxq,dyq,heat,coriolis,delsig,si,sikap,slkap,\
                                         r1b,r2b,cth1,cth2,cost_lg,kmax,imax,jmax)
    #
    #
    # Grid to spectral transformation of nlprod results
    #
    ddtdiv,ddtvort = vortdivspec(vsht,a,b,kmax,mw,zw)
    zmn3 = - ddtvort
    dmn3 = ddtdiv - lap_sht(dsht,e,mw,zw)
    _,ttend = vortdivspec(vsht,ut,vt,kmax,mw,zw)
    #
    tmn3 = -ttend + dsht(ri)
    wmn3 = dsht(wj) ### Prescribed heating converted to spectral
    qmn3 = -dsht(cbar) ### Only cbar here since dbar is included in implicit or explicit
    # 
    # Diffusion, Damping, Implicit or Explicit time differencing, Time filter
    #
    zmn3,dmn3,tmn3 = diffsn(zmn1,zmn3,dmn1,dmn3,tmn1,tmn3,mw,zw)
    #
    zmn3,dmn3,tmn3,qmn3 = damp_test(zmn1,zmn3,dmn1,dmn3,tmn1,tmn3,qmn1,qmn3,\
                          tclim,lnpsclim,vortclim,divclim,kmax,mw,zw)
    #
    dt = 86400.0/steps_per_day
    #
    zmn1,zmn2,zmn3,dmn1,dmn2,dmn3,tmn1,tmn2,tmn3,qmn1,qmn2,qmn3 = \
                        explicit(dt,amtrx,cmtrx,dmtrx,emtrx,\
                        zmn1,zmn2,zmn3,dmn1,dmn2,dmn3,tmn1,tmn2,\
                        tmn3,wmn1,wmn2,wmn3,qmn1,qmn2,qmn3,phismn,\
                        delsig,kmax,mw,zw)
    ####
    ## Reset zmn3, dmn3, tmn3,wmn3 & qmn3
    ###
    zmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
    dmn3 = zmn3.detach().clone()
    tmn3 = zmn3.detach().clone()
    wmn3 = zmn3.detach().clone()
    qmn3 = torch.zeros((mw,zw),dtype=torch.complex128)
#
# Done

(tensor(2.3246e-07-1.8869e-07j, dtype=torch.complex128), tensor(-8.1715e-08+2.6504e-07j, dtype=torch.complex128))
['Day = ', 1]
(tensor(2.1857e-07-1.6358e-07j, dtype=torch.complex128), tensor(-7.4629e-08+2.8977e-07j, dtype=torch.complex128))
['Day = ', 2]
(tensor(2.7532e-07-1.7504e-07j, dtype=torch.complex128), tensor(-8.2680e-08+3.0052e-07j, dtype=torch.complex128))
['Day = ', 3]
(tensor(2.7362e-07-1.5222e-07j, dtype=torch.complex128), tensor(-8.9963e-08+2.6365e-07j, dtype=torch.complex128))
['Day = ', 4]
(tensor(2.3433e-07-1.1309e-07j, dtype=torch.complex128), tensor(-3.6308e-08+3.0761e-07j, dtype=torch.complex128))
['Day = ', 5]
(tensor(1.8462e-07-8.6256e-08j, dtype=torch.complex128), tensor(-1.1825e-07+2.7214e-07j, dtype=torch.complex128))
['Day = ', 6]
(tensor(2.0865e-07-7.2731e-08j, dtype=torch.complex128), tensor(-8.6866e-08+2.1718e-07j, dtype=torch.complex128))
['Day = ', 7]
(tensor(2.2776e-07-6.7301e-08j, dtype=torch.complex128), tensor(-4.3913e-08+2.1283e-07j, dtype=torch.com

(tensor(1.7694e-07+8.3522e-08j, dtype=torch.complex128), tensor(-3.9851e-08+5.7956e-08j, dtype=torch.complex128))
['Day = ', 65]
(tensor(2.8389e-07+1.7136e-07j, dtype=torch.complex128), tensor(1.1683e-08+5.3813e-08j, dtype=torch.complex128))
['Day = ', 66]
(tensor(3.5391e-07+1.7085e-07j, dtype=torch.complex128), tensor(-4.0361e-08+1.0546e-07j, dtype=torch.complex128))
['Day = ', 67]
(tensor(3.3339e-07+1.0171e-07j, dtype=torch.complex128), tensor(-5.0095e-08+1.0580e-07j, dtype=torch.complex128))
['Day = ', 68]
(tensor(2.4610e-07+6.8904e-08j, dtype=torch.complex128), tensor(-2.2967e-08+8.2673e-08j, dtype=torch.complex128))
['Day = ', 69]
(tensor(2.1076e-07+4.2096e-08j, dtype=torch.complex128), tensor(-8.0306e-08+9.8549e-08j, dtype=torch.complex128))
['Day = ', 70]
(tensor(2.2290e-07+3.4337e-08j, dtype=torch.complex128), tensor(-2.5141e-08+9.4478e-08j, dtype=torch.complex128))
['Day = ', 71]
(tensor(9.2706e-08+6.0740e-08j, dtype=torch.complex128), tensor(-4.6564e-08+6.9877e-08j, dtype=tor

(tensor(1.8899e-07-7.2126e-08j, dtype=torch.complex128), tensor(-3.9379e-08+2.0081e-07j, dtype=torch.complex128))
['Day = ', 129]
(tensor(1.6947e-07+7.2248e-09j, dtype=torch.complex128), tensor(-4.5851e-08+1.8726e-07j, dtype=torch.complex128))
['Day = ', 130]
(tensor(1.5675e-07+3.5616e-08j, dtype=torch.complex128), tensor(-7.5379e-08+1.7355e-07j, dtype=torch.complex128))
['Day = ', 131]
(tensor(2.0480e-07+6.5834e-08j, dtype=torch.complex128), tensor(-6.0101e-08+2.0102e-07j, dtype=torch.complex128))
['Day = ', 132]
(tensor(2.5649e-07+2.4356e-08j, dtype=torch.complex128), tensor(-4.2280e-08+1.8812e-07j, dtype=torch.complex128))
['Day = ', 133]
(tensor(2.7267e-07-6.0950e-08j, dtype=torch.complex128), tensor(-1.0844e-07+1.7818e-07j, dtype=torch.complex128))
['Day = ', 134]
(tensor(2.5656e-07-5.7903e-08j, dtype=torch.complex128), tensor(-9.6614e-08+1.7474e-07j, dtype=torch.complex128))
['Day = ', 135]
(tensor(2.7396e-07-7.8567e-08j, dtype=torch.complex128), tensor(-8.6338e-08+1.8337e-07j, d

(tensor(3.3611e-07-1.3087e-07j, dtype=torch.complex128), tensor(-4.8351e-08+1.3619e-07j, dtype=torch.complex128))
['Day = ', 193]
(tensor(2.0517e-07-1.3183e-07j, dtype=torch.complex128), tensor(-8.1354e-08+1.5997e-07j, dtype=torch.complex128))
['Day = ', 194]
(tensor(1.6668e-07-2.5851e-08j, dtype=torch.complex128), tensor(-6.1214e-08+1.4675e-07j, dtype=torch.complex128))
['Day = ', 195]
(tensor(2.2497e-07+2.7482e-08j, dtype=torch.complex128), tensor(-3.9939e-08+1.5450e-07j, dtype=torch.complex128))
['Day = ', 196]
(tensor(3.4724e-07+5.3875e-09j, dtype=torch.complex128), tensor(-2.4099e-08+1.6824e-07j, dtype=torch.complex128))
['Day = ', 197]
(tensor(2.9903e-07-7.2363e-08j, dtype=torch.complex128), tensor(-8.5766e-08+1.3495e-07j, dtype=torch.complex128))
['Day = ', 198]
(tensor(2.0296e-07-1.1661e-07j, dtype=torch.complex128), tensor(-5.9508e-08+9.3338e-08j, dtype=torch.complex128))
['Day = ', 199]
(tensor(2.0659e-07-2.2766e-08j, dtype=torch.complex128), tensor(-5.9390e-08+1.5174e-07j, d

(tensor(2.5524e-07-2.1407e-07j, dtype=torch.complex128), tensor(-1.2633e-07+2.2157e-07j, dtype=torch.complex128))
['Day = ', 257]
(tensor(2.8935e-07-2.3651e-07j, dtype=torch.complex128), tensor(-1.5165e-07+2.6245e-07j, dtype=torch.complex128))
['Day = ', 258]
(tensor(3.0813e-07-2.9114e-07j, dtype=torch.complex128), tensor(-1.5032e-07+2.5295e-07j, dtype=torch.complex128))
['Day = ', 259]
(tensor(2.7042e-07-2.9873e-07j, dtype=torch.complex128), tensor(-8.5331e-08+2.6867e-07j, dtype=torch.complex128))
['Day = ', 260]
(tensor(2.3075e-07-2.5031e-07j, dtype=torch.complex128), tensor(-1.3153e-07+2.5824e-07j, dtype=torch.complex128))
['Day = ', 261]
(tensor(2.2827e-07-1.6472e-07j, dtype=torch.complex128), tensor(-1.1285e-07+2.8521e-07j, dtype=torch.complex128))
['Day = ', 262]
(tensor(3.2745e-07-8.9420e-08j, dtype=torch.complex128), tensor(-1.0775e-07+3.2148e-07j, dtype=torch.complex128))
['Day = ', 263]
(tensor(4.7930e-07-1.4908e-07j, dtype=torch.complex128), tensor(-1.8724e-07+2.6407e-07j, d

(tensor(3.4466e-07-1.4583e-07j, dtype=torch.complex128), tensor(-1.4901e-07+2.4781e-07j, dtype=torch.complex128))
['Day = ', 321]
(tensor(3.3845e-07-1.1262e-07j, dtype=torch.complex128), tensor(-5.8898e-08+2.4670e-07j, dtype=torch.complex128))
['Day = ', 322]
(tensor(2.9111e-07-4.9892e-08j, dtype=torch.complex128), tensor(-8.3388e-08+2.2973e-07j, dtype=torch.complex128))
['Day = ', 323]
(tensor(3.3515e-07-8.2329e-08j, dtype=torch.complex128), tensor(-1.3485e-07+2.3396e-07j, dtype=torch.complex128))
['Day = ', 324]
(tensor(3.3388e-07-1.2392e-07j, dtype=torch.complex128), tensor(-1.1836e-07+1.7860e-07j, dtype=torch.complex128))
['Day = ', 325]
(tensor(3.5715e-07-1.5460e-07j, dtype=torch.complex128), tensor(-1.4542e-07+1.7802e-07j, dtype=torch.complex128))
['Day = ', 326]
(tensor(3.2048e-07-1.6785e-07j, dtype=torch.complex128), tensor(-1.3222e-07+2.2092e-07j, dtype=torch.complex128))
['Day = ', 327]
(tensor(2.6866e-07-1.4616e-07j, dtype=torch.complex128), tensor(-1.4961e-07+2.6586e-07j, d

(tensor(3.0021e-07-2.1554e-07j, dtype=torch.complex128), tensor(-7.4078e-08+2.8060e-07j, dtype=torch.complex128))
['Day = ', 385]
(tensor(2.3943e-07-2.1685e-07j, dtype=torch.complex128), tensor(-2.6183e-08+2.8158e-07j, dtype=torch.complex128))
['Day = ', 386]
(tensor(1.9441e-07-1.6429e-07j, dtype=torch.complex128), tensor(-4.1671e-08+2.7669e-07j, dtype=torch.complex128))
['Day = ', 387]
(tensor(2.0975e-07-1.4140e-07j, dtype=torch.complex128), tensor(-1.4117e-08+2.7367e-07j, dtype=torch.complex128))
['Day = ', 388]
(tensor(2.4169e-07-1.6587e-07j, dtype=torch.complex128), tensor(-2.3510e-08+3.0512e-07j, dtype=torch.complex128))
['Day = ', 389]
(tensor(2.7058e-07-1.6209e-07j, dtype=torch.complex128), tensor(-1.6360e-08+3.0230e-07j, dtype=torch.complex128))
['Day = ', 390]
(tensor(3.2368e-07-2.0347e-07j, dtype=torch.complex128), tensor(-1.8448e-08+2.8455e-07j, dtype=torch.complex128))
['Day = ', 391]
(tensor(2.7023e-07-1.6741e-07j, dtype=torch.complex128), tensor(-1.0774e-08+2.9970e-07j, d

(tensor(3.7043e-07-1.8243e-07j, dtype=torch.complex128), tensor(-1.0576e-07+2.1564e-07j, dtype=torch.complex128))
['Day = ', 449]
(tensor(2.8370e-07-1.7296e-07j, dtype=torch.complex128), tensor(-7.8613e-08+2.5355e-07j, dtype=torch.complex128))
['Day = ', 450]
(tensor(3.1618e-07-2.1651e-07j, dtype=torch.complex128), tensor(-1.2030e-07+2.7029e-07j, dtype=torch.complex128))
['Day = ', 451]
(tensor(2.0628e-07-1.7161e-07j, dtype=torch.complex128), tensor(-1.1659e-07+2.5871e-07j, dtype=torch.complex128))
['Day = ', 452]
(tensor(1.5694e-07-1.5410e-07j, dtype=torch.complex128), tensor(-1.7175e-07+2.8469e-07j, dtype=torch.complex128))
['Day = ', 453]
(tensor(1.7520e-07-1.0975e-07j, dtype=torch.complex128), tensor(-1.7480e-07+2.7276e-07j, dtype=torch.complex128))
['Day = ', 454]
(tensor(1.6074e-07-1.3467e-07j, dtype=torch.complex128), tensor(-1.3195e-07+2.3384e-07j, dtype=torch.complex128))
['Day = ', 455]
(tensor(1.3270e-07-1.6789e-07j, dtype=torch.complex128), tensor(-1.4982e-07+2.4277e-07j, d

(tensor(1.1574e-07-1.8807e-07j, dtype=torch.complex128), tensor(-5.1974e-08+2.2620e-07j, dtype=torch.complex128))
['Day = ', 513]
(tensor(1.5358e-07-1.1716e-07j, dtype=torch.complex128), tensor(-2.3337e-08+2.3533e-07j, dtype=torch.complex128))
['Day = ', 514]
(tensor(1.8932e-07-5.5791e-08j, dtype=torch.complex128), tensor(-5.7480e-08+2.0021e-07j, dtype=torch.complex128))
['Day = ', 515]
(tensor(2.6536e-07-4.2289e-08j, dtype=torch.complex128), tensor(-7.9514e-08+2.1276e-07j, dtype=torch.complex128))
['Day = ', 516]
(tensor(2.8812e-07-6.7958e-08j, dtype=torch.complex128), tensor(-9.9148e-08+2.5832e-07j, dtype=torch.complex128))
['Day = ', 517]
(tensor(2.9357e-07-1.1353e-07j, dtype=torch.complex128), tensor(-7.0746e-08+2.1985e-07j, dtype=torch.complex128))
['Day = ', 518]
(tensor(3.1154e-07-1.6296e-07j, dtype=torch.complex128), tensor(-1.2622e-07+1.8984e-07j, dtype=torch.complex128))
['Day = ', 519]
(tensor(3.1899e-07-1.6550e-07j, dtype=torch.complex128), tensor(-9.6126e-08+2.4933e-07j, d

(tensor(1.5018e-07-3.1322e-07j, dtype=torch.complex128), tensor(-1.2926e-07+2.9426e-07j, dtype=torch.complex128))
['Day = ', 577]
(tensor(1.5437e-07-3.5377e-07j, dtype=torch.complex128), tensor(-9.9882e-08+3.3404e-07j, dtype=torch.complex128))
['Day = ', 578]
(tensor(7.0846e-08-4.0771e-07j, dtype=torch.complex128), tensor(-6.5280e-08+3.5540e-07j, dtype=torch.complex128))
['Day = ', 579]
(tensor(9.8129e-09-2.4677e-07j, dtype=torch.complex128), tensor(-1.2357e-07+2.6107e-07j, dtype=torch.complex128))
['Day = ', 580]
(tensor(3.9182e-10-1.1629e-07j, dtype=torch.complex128), tensor(-9.8117e-08+2.5584e-07j, dtype=torch.complex128))
['Day = ', 581]
(tensor(5.8366e-08-7.7320e-08j, dtype=torch.complex128), tensor(-1.1695e-07+2.7670e-07j, dtype=torch.complex128))
['Day = ', 582]
(tensor(9.9284e-08-7.8455e-08j, dtype=torch.complex128), tensor(-9.5578e-08+2.6600e-07j, dtype=torch.complex128))
['Day = ', 583]
(tensor(8.0024e-08-9.3061e-08j, dtype=torch.complex128), tensor(-1.5676e-07+2.4686e-07j, d

In [10]:
## Write spectral data for possible restart
##
torch.save(zmn1,datapath+'zmn1.spectral.pt')
torch.save(zmn2,datapath+'zmn2.spectral.pt')
torch.save(dmn1,datapath+'dmn1.spectral.pt')
torch.save(dmn2,datapath+'dmn2.spectral.pt')
torch.save(tmn1,datapath+'tmn1.spectral.pt')
torch.save(tmn2,datapath+'tmn2.spectral.pt')
torch.save(qmn1,datapath+'qmn1.spectral.pt')
torch.save(qmn2,datapath+'qmn2.spectral.pt')